In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from entsoe import EntsoePandasClient

entsoe_client = EntsoePandasClient(api_key="API_KEY")

COUNTRY_CODE = "NL"
TIMEZONE = "Europe/Amsterdam"
DATE = pd.Timestamp("2023-11-13", tz=TIMEZONE)
END_DATE = DATE.replace(hour=23, minute=59)

sns.set_theme()
sns.set(context="notebook", font="Verdana", font_scale=1.5)
matplotlib.rcParams['figure.figsize'] = (16, 9)
matplotlib.rcParams['figure.dpi'] = 200
matplotlib.rcParams['lines.linewidth'] = 2.5

print("Default figure size:", matplotlib.rcParams['figure.figsize'])

In [ ]:
def calculate_synthetic_values(df: pd.DataFrame) -> pd.DataFrame:
    df["Total renewable generation"] = df["Solar"] + df["Wind Offshore"] + df["Wind Onshore"]
    df["Renewable percentage"] = df["Total renewable generation"] / df["Forecasted Load"] * 100
    
    return df

In [ ]:
renewable_forecast = entsoe_client.query_wind_and_solar_forecast(
    country_code=COUNTRY_CODE,
    start=DATE,
    end=END_DATE,
)
load_forecast = entsoe_client.query_load_forecast(
    country_code=COUNTRY_CODE,
    start=DATE,
    end=END_DATE,
)

df = renewable_forecast.merge(load_forecast, left_index=True, right_index=True, validate="one_to_one")

df = calculate_synthetic_values(df)

df

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(16,4), sharey=True, sharex=True)
lines = [
    "Solar",
    "Wind Offshore",
    "Wind Onshore",
]

palette = itertools.cycle(sns.color_palette())

for line_name in lines:
    color = next(palette)
    sns.lineplot(data=df, x=df.index, y=line_name, ax=axs[0], errorbar=None, color=color)

axs[0].legend(lines)
    
sns.lineplot(data=df, x=df.index, y="Total renewable generation", ax=axs[1], errorbar=None, color=next(palette))
axs[1].legend(["Total renewable generation"])

xticks = pd.date_range(start=DATE, end=DATE + pd.Timedelta(days=1), periods=13, tz=TIMEZONE)

for ax in axs:
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticks.strftime("%H:%M"), rotation=45, horizontalalignment="right")
    ax.set_ylabel("MW")
    ax.set_xlabel("Time of day (November 13th, 2023")
    ax.yaxis.set_major_formatter('{x:,.0f}')